In [3]:
pip install --upgrade hats-import hats nested-pandas lsdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2025.2.0
    Uninstalling dask-2025.2.0:
      Successfully uninstalled dask-2025.2.0
  Attempting uninstall: distributed
    Found existing installation: distributed 2025.2.0
    Uninstalling distributed-2025.2.0:
      Successfully uninstalled distributed-2025.2.0
  Attempting uninstall: lsdb
    Found existing installation: lsdb 0.4.6
    Uninstalling lsdb-0.4.6:
      Successfully uninstalled lsdb-0.4.6
  Attempting uninstall: hats-import
    Found existing installation: hats-import 0.5.2
    Uninstalling hats-import-0.5.2:
      Successfully uninstalled hats-import-0.5.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nested-dask 0.3.4 req

In [3]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import nested_pandas as npd
import pyarrow as pa
import numpy as np

import tarfile
import glob

In [21]:
# untarring
filepath = "/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/"

ss_tarfiles = glob.glob(filepath+"ss_tar_26/*.tar.gz")


for ss_tarfile in ss_tarfiles:
    file = tarfile.open(ss_tarfile) 
  
    # extracting file
    for member in file.getmembers():
        if "spPlate" in member.name:
            file.extract(member, filepath+"ss_26/")
#file.extractall(filepath+"ss_26/") 
  
    file.close()


In [20]:
# parquet-ifying
filepath = "/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26/"
run_path = "/data/epyc/users/brantd/sdss_dl_testing"

ss_fitsfiles = glob.glob(filepath+"/*/*.fits")

for ss_fitsfile in ss_fitsfiles:
    obj_dat = Table.read(ss_fitsfile, hdu=5, format='fits')
    obj_dat.write(f'{run_path}/h5_pq/plate.parquet', format='parquet', overwrite=True)
    #obj_dat.to_pandas().to_parquet('h5_pq/plate.parquet')

    # Read in spFrame HDU data
    #with pd.read_parquet('h5_pq/plate.parquet', dtype_backend='pyarrow') as ndf:
    ndf = npd.read_parquet(f'{run_path}/h5_pq/plate.parquet')#, dtype_backend='pyarrow')
    #ndf = ndf.assign(OBJID_joined = lambda x: "".join(val for val in x["OBJID"]))
    #joined_objid = ndf.apply(lambda x: "".join([str(val) for val in x.OBJID.tolist()]), axis=1).astype("int")
    #ndf["joined_OBJID"] = joined_objid

    ndf = ndf.astype({"OBJID": pd.ArrowDtype(pa.list_(pa.int32())),
                      "MAG": pd.ArrowDtype(pa.list_(pa.float32())),
                      "CALIBFLUX": pd.ArrowDtype(pa.list_(pa.float32())),
                      "CALIBFLUX_IVAR": pd.ArrowDtype(pa.list_(pa.float32()))})
    with fits.open(ss_fitsfile) as spec:
        ndf["flux"] = pd.Series(spec[0].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["ivar"] = pd.Series(spec[1].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["andmask"] = pd.Series(spec[2].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
        ndf["ormask"] = pd.Series(spec[3].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
        ndf["wavedisp"] = pd.Series(spec[4].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["skyflux"] = pd.Series(spec[6].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))

    ndf = npd.NestedFrame.from_lists(ndf, list_columns=["flux", "ivar", "andmask", "ormask", "wavedisp", "skyflux"], name="spectra")
    ndf = ndf.query("RA >=-90 and DEC >=-90")
    ndf.to_parquet(f"/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/{ss_fitsfile.split('/')[-1].split('.')[0]}.parquet", 
    compression="snappy", store_schema=False)






        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 1
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]


In [18]:
npd.read_parquet("/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/spPlate-0616-52374.parquet").RA.min()

-9999.0

In [29]:
#npd.read_parquet("/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/spPlate-0266-51602.parquet", dtype_backend="pyarrow")
#pd.read_parquet("h5_pq/plate.parquet", dtype_backend="pyarrow").dtypes

In [1]:
#hats importing
from hats_import.catalog.arguments import ImportArguments
import hats_import.pipeline as runner
from dask.distributed import Client
from hats_import.pipeline import pipeline_with_client

args = ImportArguments(
    output_artifact_name="sdss_dr7_spectra",
    input_path="/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/",
    file_reader="parquet",
    ra_column="RA",
    dec_column="DEC",
    #sort_columns="joined_OBJID",
    pixel_threshold=2500,
    #highest_healpix_order=7,
    output_path="/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_catalog_nested",
)
#runner.pipeline(args)
# Run the import pipeline
with Client(n_workers=10, threads_per_worker=1, memory_limit="10GB") as client:
    # with a single thread, the memory_limit here may be overly conservative, feel free to experiment with reducing it
    pipeline_with_client(args, client)

/astro/users/brantd/.conda/envs/lsdb_latest/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/astro/users/brantd/.conda/envs/lsdb_latest/lib/python3.11/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41244 instead
  warnings.warn(
Finishing : 100%|██████████| 5/5 [00:02<00:00,  2.15it/s]


In [2]:
import lsdb
import nested_pandas
(lsdb.__version__, nested_pandas.__version__)

('0.4.6', '0.4.3')

In [1]:
# Validation
from lsdb import read_hats

read_hats("/astro/store/epyc3/data3/hats/catalogs/sdss_dr7/sdss_dr7_spectra").head(10)

OBJID   HOLETYPE         RA       DEC  \
_healpix_29                                                                 
168688995934        [125   7   4 375 284]  b'OBJECT'   44.99191  0.040806   
261643454890   [2709    0    4  168  203]  b'OBJECT'  44.997798  0.063088   
393536217765        [125   4   4 375 718]  b'OBJECT'   45.05074  0.078381   
393536217785        [125   4   4 375 718]  b'OBJECT'   45.05074  0.078381   
468807921113   [2709    0    4  169   95]  b'OBJECT'  45.026711  0.081701   
566623584950        [125   7   4 375 218]  b'OBJECT'  44.955173  0.054207   
665282407490   [3325   41    4  408  136]  b'OBJECT'  44.961071  0.077109   
739458918010        [125   7   4 375 115]  b'OBJECT'  44.931922  0.077136   
881518682163   [3325   41    4  408  159]  b'OBJECT'  45.005005  0.102012   
1020156509006  [3325   41    4  408  480]  b'OBJECT'  44.981201   0.11988   

                                                     MAG        STARL  \
_healpix_29                                                             
168688995934   [24.124  22.5036 21.0946 20.2709 19.9048]          0.0   
261643454890   [21.5871 21.0464 20.3375 20.1342 19.9286]          0.0   
393536217765   [24.2274 24.451  25.404  24.3942 22.4883]       0.7482   
393536217785   [24.2274 24.451  25.404  24.3942 22.4883]     0.748169   
468807921113             [23.39 20.98 19.6  18.94 18.65]          0.0   
566623584950   [22.5278 21.6577 20.8072 20.2478 19.986 ]          0.0   
665282407490        [21.813 20.686 20.183 19.764 19.713]          0.0   
739458918010   [20.7897 19.9775 19.7239 19.357  19.4774]  150856000.0   
881518682163             [23.56 21.02 19.57 18.77 18.37]          0.0   
1020156509006       [23.652 22.958 21.739 21.002 20.437]     0.005741   

                      EXPL      DEVAUCL                 OBJTYPE     XFOCAL  \
_healpix_29                                                                  
168688995934           0.0          0.0     b'GALAXY          ' -173.12304   
261643454890           0.0          0.0     b'QSO             '  247.47207   
393536217765        0.7079       0.6516     b'SKY             ' -160.29831   
393536217785      0.707946     0.651628     b'SKY             ' -160.29841   
468807921113           0.0          0.0     b'QSO             '  253.78466   
566623584950           0.0          0.0     b'GALAXY          ' -181.13305   
665282407490      0.952796          0.0     b'GALAXY          '  239.44266   
739458918010   150856000.0  150856000.0     b'GALAXY          ' -186.20325   
881518682163           0.0          0.0  b'QSO                ' -170.25908   
1020156509006     0.071945     0.195884     b'GALAXY          '  243.83677   

               ...  THROUGHPUT   PRIMTARGET    SECTARGET  OFFSETID  \
_healpix_29    ...                                                   
168688995934   ...        3997  -2147483584  -2147483648         1   
261643454890   ...        5584  -2147483646  -2147483648         1   
393536217765   ...        3818            0           16         1   
393536217785   ...        3504            0           16         1   
468807921113   ...        3241  -1610612736  -2147483648         1   
566623584950   ...        4619  -2147483584  -2147483648         1   
665282407490   ...        1708  -2147483584  -2147483648         1   
739458918010   ...        3172  -2147483584  -2147483648         1   
881518682163   ...        3762  -2147483646  -2147483648         1   
1020156509006  ...        2033  -2147483584  -2147483648         1   

               SCI_EXPTIME                                          CALIBFLUX  \
_healpix_29                                                                     
168688995934           0.0  [ 0.26300535  1.1792095   4.6411023   9.163324...   
261643454890           0.0  [ 2.1954396  3.637493   6.5593147  7.866824  1...   
393536217765           0.0  [0.08319576 0.09556487 0.05375066 0.08982822 0...   
393536217785           0.0  [0.08319576 0.09556487 

In [4]:
# NEW -- use header 5 data as object table, store all spectral data as list-arrays
run_path = "/data/epyc/users/brantd/sdss_dl_testing"

# Read Header 5 - 640 rows
obj_dat = Table.read("spectro/ss_26/0666/spPlate-0666-52149.fits", hdu=5, format='fits')
obj_dat.write('h5_pq/plate.parquet', format='parquet', overwrite=True)

# Read in spFrame HDU data
ndf = pd.read_parquet('h5_pq/plate.parquet', dtype_backend='pyarrow')
ndf["flux"] = pd.Series(spec[0].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["ivar"] = pd.Series(spec[1].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["andmask"] = pd.Series(spec[2].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
ndf["ormask"] = pd.Series(spec[3].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
ndf["wavedisp"] = pd.Series(spec[4].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["skyflux"] = pd.Series(spec[6].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))

ndf


FileNotFoundError: [Errno 2] No such file or directory: 'spectro/ss_26/0666/spPlate-0666-52149.fits'

In [5]:
#OLD
run_path = "/data/epyc/users/brantd/sdss_dl_testing"
# Read Object Table - 640 rows
obj_dat = Table.read(f"{run_path}/spectro/ss_26/0666/spObj-0666-52149-26.fit", hdu=1, format='fits')

#Convert to NestedFrame
obj_cols = ["run", "camcol", "rerun", "field", "parent", "id", "nchild", 
"ra","dec", "lambda", "eta", "l","b", "primtarget", "sectarget"]
ndf = npd.NestedFrame(obj_dat[obj_cols].to_pandas())

# Read in spFrame HDU data
with fits.open(f"{run_path}/spectro/ss_26/0666/spPlate-0666-52149.fits") as spec:
    ndf["flux"] = pd.Series(spec[0].data.tolist())
    ndf["ivar"] = pd.Series(spec[1].data.tolist())
    ndf["andmask"] = pd.Series(spec[2].data.tolist())
    ndf["ormask"] = pd.Series(spec[3].data.tolist())
    ndf["wavedisp"] = pd.Series(spec[4].data.tolist())
    ndf["skyflux"] = pd.Series(spec[6].data.tolist())

ndf_spec = npd.NestedFrame.from_lists(ndf, base_columns=obj_cols, name="spectra")
ndf_spec

run  camcol  rerun  field  parent   id  nchild         ra        dec  \
0    1729       2     40    290     166  167       0  31.523135  -9.756108   
1    1729       1     40    289      -1  103       0  31.469417 -10.083744   
..    ...     ...    ...    ...     ...  ...     ...        ...        ...   
638  1729       4     40    288     180  181       0  31.158739  -8.956450   
639  1729       4     40    291      90   91       0  31.584228  -8.882637   

         lambda        eta           l          b  primtarget  sectarget  \
0   -153.889674 -21.622311  171.850705 -65.266138          64          0   
1   -153.970429 -21.263943  172.295789 -65.541507           2          0   
..          ...        ...         ...        ...         ...        ...   
638 -154.183893 -22.540958  169.944968 -64.913311          32          0   
639 -153.759129 -22.587009  170.568554 -64.579597    33554496          0   

                                               spectra  
0    [{flux: 9.425934, ivar: 0.0, andmask: 16777216...  
1    [{flux: 10.505326, ivar: 0.0, andmask: 1677721...  
..                                                 ...  
638  [{flux: 0.871144, ivar: 0.0, andmask: 16777216...  
639  [{flux: 15.76809, ivar: 0.0, andmask: 16777216...  

[640 rows x 16 columns]

In [16]:
ndf_spec["spectra"].iloc[0]

,flux,ivar,andmask,ormask,wavedisp,skyflux
0,9.425934,0.000000,16777216,16777216,0.000000,0.000000
1,9.426754,0.000000,16777216,16777216,0.000000,0.000000
2,9.427573,0.000000,16777216,16777216,0.000000,0.000000
3,9.428390,0.000000,16777216,16777216,0.000000,0.000000
4,9.429207,0.000000,16777216,16777216,0.000000,0.000000
...,...,...,...,...,...,...
3844,14.376133,1.317432,0,0,0.712094,5.603570
3845,13.377356,1.356143,0,0,0.712099,5.916445
3846,14.397409,0.000000,0,0,0.711992,6.316115
3847,15.417425,1.253372,0,0,0.712273,6.734169


In [5]:
ndf_spec.to_parquet("./plate_test", by_layer=True)

In [6]:
obj = fits.open("spectro/ss_26/0666/spObj-0666-52149-26.fit")
spec = fits.open("spectro/ss_26/0666/spPlate-0666-52149.fits")

In [12]:
spec[5].data[:2]

FITS_rec([([1729,    0,    2,  290,   91], 'OBJECT', 31.523155,  -9.7561011, [23.2484, 19.117 , 18.2746, 17.746 , 17.6056], 0.      , 0.      , 2.22331e-08, 'GALAXY', 199.74945, -163.01831, 1, 1, 2230, 64, 0, 1, 0., [  5.404982,  21.817234,  77.35365 , 112.39978 , 157.49948 ], [4.0015516 , 0.        , 0.6940962 , 0.24724363, 0.08935539], 0.02303132),
          ([1729,    0,    1,  289,  132], 'OBJECT', 31.469403, -10.083748 , [19.8646, 19.5346, 19.23  , 19.2221, 19.2284], 0.737904, 0.302691, 3.47536e-01, 'QSO', 188.15113, -234.50142, 1, 2, 2224,  4, 0, 1, 0., [ 15.067631,  17.1941  ,  23.961672,  23.531322,  24.633608], [4.6041636 , 9.9697075 , 3.592663  , 5.5618854 , 0.6966781 ], 0.0212433 )],
         dtype=(numpy.record, [('OBJID', '>i4', (5,)), ('HOLETYPE', 'S6'), ('RA', '>f8'), ('DEC', '>f8'), ('MAG', '>f4', (5,)), ('STARL', '>f4'), ('EXPL', '>f4'), ('DEVAUCL', '>f4'), ('OBJTYPE', 'S16'), ('XFOCAL', '>f8'), ('YFOCAL', '>f8'), ('SPECTROGRAPHID', '>i4'), ('FIBERID', '>i4'), ('THROUG

In [8]:
np.shape(spec[5].data)

(640,)

In [66]:
spec[0].data[0]

array([ 9.425934,  9.426754,  9.427573, ..., 14.397409, 15.417425,
       15.417142], dtype='>f4')

In [89]:
import matplotlib.pyplot as plt

len(spec[0].data[0])

3849

In [88]:
#Table.read.help('fits')

In [129]:
len(obj[1].data)

640

In [23]:
spec_dat = Table.read("spectro/ss_26/0666/spPlate-0666-52149.fits", hdu=5, format='fits')
spec_dat[:2].to_pandas()

ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['OBJID', 'MAG', 'CALIBFLUX', 'CALIBFLUX_IVAR']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)

In [22]:
spec[5].data[:2]

FITS_rec([([1729,    0,    2,  290,   91], 'OBJECT', 31.523155,  -9.7561011, [23.2484, 19.117 , 18.2746, 17.746 , 17.6056], 0.      , 0.      , 2.22331e-08, 'GALAXY', 199.74945, -163.01831, 1, 1, 2230, 64, 0, 1, 0., [  5.404982,  21.817234,  77.35365 , 112.39978 , 157.49948 ], [4.0015516 , 0.        , 0.6940962 , 0.24724363, 0.08935539], 0.02303132),
          ([1729,    0,    1,  289,  132], 'OBJECT', 31.469403, -10.083748 , [19.8646, 19.5346, 19.23  , 19.2221, 19.2284], 0.737904, 0.302691, 3.47536e-01, 'QSO', 188.15113, -234.50142, 1, 2, 2224,  4, 0, 1, 0., [ 15.067631,  17.1941  ,  23.961672,  23.531322,  24.633608], [4.6041636 , 9.9697075 , 3.592663  , 5.5618854 , 0.6966781 ], 0.0212433 )],
         dtype=(numpy.record, [('OBJID', '>i4', (5,)), ('HOLETYPE', 'S6'), ('RA', '>f8'), ('DEC', '>f8'), ('MAG', '>f4', (5,)), ('STARL', '>f4'), ('EXPL', '>f4'), ('DEVAUCL', '>f4'), ('OBJTYPE', 'S16'), ('XFOCAL', '>f8'), ('YFOCAL', '>f8'), ('SPECTROGRAPHID', '>i4'), ('FIBERID', '>i4'), ('THROUG

In [79]:
spec.info()

Filename: spectro/ss_26/0666/spPlate-0666-52149.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     199   (3849, 640)   float32   
  1                1 ImageHDU        15   (3849, 640)   float32   
  2                1 ImageHDU        14   (3849, 640)   int32   
  3                1 ImageHDU        14   (3849, 640)   int32   
  4                1 ImageHDU        15   (3849, 640)   float32   
  5                1 BinTableHDU     59   640R x 21C   ['5J', '6A', 'D', 'D', '5E', 'E', 'E', 'E', '16A', 'D', 'D', 'J', 'J', 'J', 'J', 'J', 'J', 'E', '5E', '5E', 'E']   
  6                1 ImageHDU         7   (3849, 640)   float32   
  7                1 ImageHDU         7   (5, 640)   float32   
  8                1 ImageHDU         7   (3, 640)   float32   


In [133]:
spec[0].header

SIMPLE  =                    T / FIT STANDARD                                   
BITPIX  =                  -32 / FITS BITS/PIXEL                                
NAXIS   =                    2 / NUMBER OF AXES                                 
NAXIS1  =                 3849 / WIDTH (TOTAL!)                                 
NAXIS2  =                  640 / HEIGHT (TOTAL!)                                
EXTEND  =                    T /                                                
TAI     =        4505617383.37 / 1st row - Number of seconds since Nov 17 1858  
RA      =            30.592416 / 1st row - Right ascension of telescope boresigh
DEC     =           -9.008748  / 1st row - Declination of telescope boresight (d
EQUINOX =              2000.00 /                                                
RADECSYS= 'FK5     '           /                                                
AZ      =        272.914059222 / 1st row - Azimuth  (encoder) of tele (0=N?) (de
ALT     =        47.63957600

In [123]:
len(spec[6].data[0])

3849

In [132]:
obj_dat = Table.read("spectro/ss_26/0666/spObj-0666-52149-26.fit", hdu=1, format='fits')
obj_dat[0:2]

run,camcol,rerun,field,parent,id,nchild,objc_type,objc_prob_psf,catid,objc_flags,objc_flags2,objc_rowc,objc_rowcerr,objc_colc,objc_colcerr,rowv,rowverr,colv,colverr,rowc,rowcerr,colc,colcerr,sky,skyerr,psfcounts,psfcountserr,fibercounts,fibercountserr,petrocounts,petrocountserr,petrorad,petroraderr,petror50,petror50err,petror90,petror90err,q,qerr,u,uerr,m_e1,m_e2,m_e1e1err,m_e1e2err,m_e2e2err,m_rr_cc,m_rr_ccerr,m_cr4,m_e1_psf,m_e2_psf,m_rr_cc_psf,m_cr4_psf,iso_rowc,iso_rowcerr,iso_rowcgrad,iso_colc,iso_colcerr,iso_colcgrad,iso_a,iso_aerr,iso_agrad,iso_b,iso_berr,iso_bgrad,iso_phi,iso_phierr,iso_phigrad,r_dev,r_deverr,ab_dev,ab_deverr,phi_dev,phi_deverr,counts_dev,counts_deverr,r_exp,r_experr,ab_exp,ab_experr,phi_exp,phi_experr,counts_exp,counts_experr,counts_model,counts_modelerr,texture,star_l,star_lnl,exp_l,exp_lnl,dev_l,dev_lnl,fracpsf,flags,flags2,type,prob_psf,nprof,profmean,proferr,status,ra,dec,lambda,eta,l,b,offsetra,offsetdec,primtarget,sectarget,reddening,propermotionmatch,propermotiondelta,propermotion,propermotionangle,usnoblue,usnored,firstmatch,firstid,firstlambda,firsteta,firstdelta,firstpeak,firstint,firstrms,firstmajor,firstminor,firstpa,rosatmatch,rosatdelta,rosatposerr,rosatcps,rosatcpserr,rosathr1,rosathr1err,rosathr2,rosathr2err,rosatext,rosatextlike,rosatdetectlike,rosatexposure,priority,matchid,sprerun,vers_2d,vers_1d,plate,mjd,mapid,fiberid,objid,diag_primtarget,diag_sectarget,xfocal,yfocal,diag_ra,diag_dec,skyres,ugrizfibre,griflux,grisn,deriv2,mean,contchi2,blueslope,redslope,class,zstat,zfinal,zerrfinal,zconffinal,zxcorr,zerrxcorr,zconfxcorr,zemz,zerremz,zconfemz,templateid,nemline,zwarning,galclass,vel_dis,vel_dise,eclass,ecoeff1,ecoeff2,ecoeff3,ecoeff4,ecoeff5,plugmapobj_objid,holetype,plugmapobj_ra,plugmapobj_dec,mag,starl,expl,devaucl,objtype,plugmapobj_xfocal,plugmapobj_yfocal,spectrographid,plugmapobj_fiberid,throughput,plugmapobj_primtarget,plugmapobj_sectarget
int32,int32,int32,int32,int32,int32,int32,int32,float32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],int32[5],int32[5],int32[5],float32[5],int32[5],"float32[5,15]","float32[5,15]",int32,float64,float64,float64,float64,float64,float64,float32[5],float32[5],int32,int32,float32[5],int32,float32,float32,float32,float32,float32,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,"int32[10,5]",int32,bytes19,bytes19,int32,int32,bytes19,int32,int32[5],int32,int32,float64,float64,float64,float64,float32[3],float32[5],float32[3],float32[3],float32,float32,float32,float32,float32,bytes80,bytes80,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,float32,float64,float64,float64,float64,float64,float64,float64,float64,int32[5],bytes80,float64,float64,float32[5],float32,float32,float32,bytes80,float64,float64,int32,int32,int32,int32,int32
1729,2,40,290,166,167,0,3,0.0,0,268566544,36864,755.6808,0.01255912,543.89215,0.019416926,0.0032394822,0.0058910768,0.0018419423,0.005908007,760.37775 .. 761.90576,0.2198638 .. 0.033098206,547.1